In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow
from  keras.models import Sequential, Model
from  keras.layers import Input, Dense, LeakyReLU, Activation, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, InputLayer, Flatten, BatchNormalization, Reshape, Lambda

from  keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from  keras.applications.vgg16 import VGG16, preprocess_input
#from  keras.applications import ResNet152V2,ResNet101V2,ResNet50V2 
#from  keras.applications.resnet_v2 import preprocess_input 

from  keras.optimizers import RMSprop, Adam
from  keras.preprocessing.image import load_img, ImageDataGenerator
from  keras.utils import multi_gpu_model
from IPython.display import Image 

import keras_metrics
import pandas
import ast
import numpy as np
import matplotlib.patches as patches 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageDraw

Using TensorFlow backend.


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras.backend as K
import numpy as np
import random

In [4]:
def yolf_loss(y_true, y_pred):
    #true e pred sono [32][245]
    
    #tensorflow.print("tensors:", y_pred)
    #tensorflow.print("batch_size:", tensorflow.shape(y_pred)[0])
    
    y_pred = tensorflow.reshape(y_pred, [tensorflow.shape(y_pred)[0], 7, 7, 5])
    y_true = tensorflow.reshape(y_true, [tensorflow.shape(y_pred)[0], 7, 7, 5])


    b_p_pred = y_pred[..., 0]
    b_x_pred = y_pred[..., 1]
    b_y_pred = y_pred[..., 2]
    b_w_pred = y_pred[..., 3]
    b_h_pred = y_pred[..., 4]


    b_p = y_true[..., 0]
    b_x = y_true[..., 1]
    b_y = y_true[..., 2]
    b_w = y_true[..., 3]
    b_h = y_true[..., 4]

    #print(b_xy_pred.get_shape(),b_xy.get_shape())
    #print(b_wh_pred.get_shape(),b_wh.get_shape())
    #indicator_coord = K.expand_dims(y_true[..., 3], axis=-1) * 1.0
    loss_p =K.sum(100*K.square(b_p - b_p_pred), axis=-1)
    loss_xy = K.sum(b_p * (K.square(b_x - b_x_pred) + K.square(b_y - b_y_pred)), axis=-1)# * indicator_coord)#, axis=[1,2,3,4])
    
    b_w = K.pow(b_w, 0.5)
    b_h = K.pow(b_h, 0.5)
    b_w_pred = K.pow(b_w_pred, 0.5)
    b_h_pred = K.pow(b_h_pred, 0.5)
    
    loss_wh = K.sum(
        b_p * 
        (
            (K.square(b_w - b_w_pred)) + 
            (K.square(b_h - b_h_pred))
        ), axis=-1)# * indicator_coord)#, axis=[1,2,3,4])

    #tensorflow.print("loss_p:", loss_p)
    #tensorflow.print("loss_xy:", loss_xy)
    #tensorflow.print("loss_wh:", loss_wh)

    #print(K.cast(loss_p, dtype="float32"))
    #print(K.cast(loss_xy, dtype="float32"))
    #print(loss_wh)
    #tensorflow.print("Loss:", ( K.cast(loss_p, dtype="float32") + loss_wh +  K.cast(loss_xy, dtype="float32") )/3)
    return (K.cast(loss_p, dtype="float32") + loss_wh +  K.cast(loss_xy, dtype="float32"))/3

In [5]:
BATCH_SIZE = 8
IMG_SIZE = 224

In [6]:
train_set = "./dataset/NUOVO/train/"
validation_set = "./dataset/NUOVO/val/"
test_set = "./dataset/NUOVO/test/"

In [7]:
train_df = pandas.read_csv("./dataset/NUOVO/train/output_final.csv")
valid_df = pandas.read_csv("./dataset/NUOVO/val/output_final.csv")
test_df = pandas.read_csv("./dataset/NUOVO/test/output_final.csv")

In [8]:
train_datagen = ImageDataGenerator(
    #rescale=1./255,
    preprocessing_function = preprocess_input)

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=train_set,
        x_col="image_id",
        y_col=["x_1","y_1","width","height"],
        class_mode="raw",
        shuffle=True,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

valid_generator = train_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory=validation_set,
        x_col="image_id",
        y_col=["x_1","y_1","width","height"],
        class_mode="raw",
        shuffle=True,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

Found 164105 validated image filenames.
Found 18234 validated image filenames.


In [9]:
step_size_train = train_generator.samples/train_generator.batch_size
step_size_valid = valid_generator.samples/valid_generator.batch_size

In [10]:
darknetv1 = Sequential()
darknetv1.add(InputLayer(input_shape=(IMG_SIZE,IMG_SIZE,3)))
darknetv1.add(Conv2D(64,3, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(MaxPooling2D(pool_size=(2, 2)))
darknetv1.add(Conv2D(192,3, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(MaxPooling2D(pool_size=(2, 2)))

darknetv1.add(Conv2D(128,1, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(Conv2D(256,3, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(Conv2D(256,1, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(Conv2D(512,3, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(MaxPooling2D(pool_size=(2, 2)))

darknetv1.add(Conv2D(256,1, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(Conv2D(512,3, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(Conv2D(256,1, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(Conv2D(512,3, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(Conv2D(256,1, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(Conv2D(512,3, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(Conv2D(256,1, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(Conv2D(512,3, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(Conv2D(512,1, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(Conv2D(1024,3, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(MaxPooling2D(pool_size=(2, 2)))

darknetv1.add(Conv2D(512,1, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(Conv2D(1024,3, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(Conv2D(512,1, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(Conv2D(1024,3, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(GlobalAveragePooling2D())

darknetv1.add(Dense(512, activation = "relu"))
darknetv1.add(Dense(512, activation = "relu"))
darknetv1.add(Dense(1, activation = "sigmoid"))

In [11]:
darknetv1.load_weights("face_classifier_BN.h5") 

# Remove last layers

In [12]:
for i in range(4):
    darknetv1.pop()

In [13]:
darknetv1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 224, 224, 64)      256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 224, 224, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 192)     110784    
_________________________________________________________________
batch_normalization_2 (Batch (None, 112, 112, 192)     768       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 112, 112, 192)    

# New layers

In [14]:
darknetv1.add(Conv2D(1024,3, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(Conv2D(1024,3, strides=(2,2), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(Conv2D(1024,3, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(Conv2D(1024,3, strides=(1,1), padding = "same"))
darknetv1.add(BatchNormalization())
darknetv1.add(LeakyReLU(alpha=0.1))
darknetv1.add(Conv2D(5,7, strides=(1,1), padding = "same", activation="relu"))
darknetv1.add(Flatten())
darknetv1.add(Dense(120, activation="relu"))
darknetv1.add(Dense(4, activation="relu"))

In [15]:
darknetv1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 224, 224, 64)      256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 224, 224, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 192)     110784    
_________________________________________________________________
batch_normalization_2 (Batch (None, 112, 112, 192)     768       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 112, 112, 192)    

In [16]:
darknetv1.compile(optimizer=Adam(lr = 1e-4), loss="mse", metrics=["mse"])

darknetv1.evaluate_generator(valid_generator, steps=step_size_valid, verbose = 1)

In [17]:
#earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
mcp_save = ModelCheckpoint('darknet_boh.h5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='min')

history = darknetv1.fit_generator(generator=train_generator, epochs=3, steps_per_epoch=step_size_train, validation_data=valid_generator, validation_steps=step_size_valid, verbose=1, callbacks=[mcp_save, reduce_lr_loss])


Epoch 1/3
20514/20513 [==============================] - 3785s 185ms/step - loss: 85.5186 - mse: 85.5217 - val_loss: 13.3235 - val_mse: 76.2083
Epoch 2/3
20514/20513 [==============================] - 3773s 184ms/step - loss: 41.0087 - mse: 41.0089 - val_loss: 23.1543 - val_mse: 41.2854
Epoch 3/3
20514/20513 [==============================] - 3784s 184ms/step - loss: 31.2742 - mse: 31.2748 - val_loss: 26.3602 - val_mse: 41.1226


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training accuracy')
plt.plot(val_acc, label='Validation accuracy')
plt.legend(loc='lower right')
plt.ylabel('accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
model_json = darknetv1.to_json()
with open("darknet_ev.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
#model.save_weights("resnet50.h5")
print("Saved model to disk")

# Loading model

In [19]:
darknetv2 = Sequential()
darknetv2.add(InputLayer(input_shape=(IMG_SIZE,IMG_SIZE,3)))
darknetv2.add(Conv2D(64,3, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(MaxPooling2D(pool_size=(2, 2)))
darknetv2.add(Conv2D(192,3, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(MaxPooling2D(pool_size=(2, 2)))

darknetv2.add(Conv2D(128,1, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(Conv2D(256,3, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(Conv2D(256,1, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(Conv2D(512,3, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(MaxPooling2D(pool_size=(2, 2)))

darknetv2.add(Conv2D(256,1, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(Conv2D(512,3, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(Conv2D(256,1, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(Conv2D(512,3, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(Conv2D(256,1, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(Conv2D(512,3, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(Conv2D(256,1, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(Conv2D(512,3, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(Conv2D(512,1, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(Conv2D(1024,3, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(MaxPooling2D(pool_size=(2, 2)))

darknetv2.add(Conv2D(512,1, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(Conv2D(1024,3, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(Conv2D(512,1, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(Conv2D(1024,3, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(Conv2D(1024,3, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(Conv2D(1024,3, strides=(2,2), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(Conv2D(1024,3, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(Conv2D(1024,3, strides=(1,1), padding = "same"))
darknetv2.add(BatchNormalization())
darknetv2.add(LeakyReLU(alpha=0.1))
darknetv2.add(Conv2D(5,7, strides=(1,1), padding = "same", activation="relu"))
darknetv2.add(Flatten())
darknetv2.add(Dense(120, activation="relu"))
darknetv2.add(Dense(4, activation="relu"))

In [20]:
darknetv2.load_weights("darknet_boh.h5") 
darknetv2.compile(optimizer=Adam(lr = 1e-4), loss="binary_crossentropy", metrics=["accuracy"])

In [21]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory=validation_set,
        x_col="image_id",
        y_col=train_df.columns[1:],
        class_mode="raw",
        shuffle=True,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)
STEP_SIZE_TEST = test_generator.n / test_generator.batch_size 


#CHANGE PARALLEL MODEL
pred=darknetv2.predict_generator(test_generator,  steps=STEP_SIZE_TEST,  verbose=1)

Found 18234 validated image filenames.
2280/2279 [==============================] - 124s 54ms/step


In [22]:
print(pred[0])

[ 50.615192  51.078506  73.974106 102.45308 ]


In [24]:
img = Image.open(validation_set+"\\"+test_generator.filenames[0])
img1 = ImageDraw.Draw(img)
x1,y1 = pred[0][0],pred[0][1]
x4,y4= pred[0][0]+pred[0][2],pred[0][1]+pred[0][3]
img1.rectangle([(x1,y1),(x4,y4)], outline ="red")
img.show() 

In [28]:
count = 0
for el in zip(pred,test_generator.labels,test_generator.filenames):
    count = count +1
    img = Image.open(validation_set+"\\"+el[2])
    img1 = ImageDraw.Draw(img)
    x1,y1 = el[0][0],el[0][1]
    x4,y4= el[0][0]+el[0][2],el[0][1]+el[0][3]
    img1.rectangle([(x1,y1),(x4,y4)], outline ="red")
    img.show() 
    if count == 1000:
        pass

KeyboardInterrupt: 